# Cenário:
Em nosso exemplo iremos trabalhar com a API aberta oferecida pelo INPE que obtém dados da
Previsão de Tempo, IUV e Ondas em XML.
A API  pode  ser acessada  pelo link:
http://servicos.cptec.inpe.br/XML/
Realize quantas e da forma que conseguir as tarefas a seguir:


# Tarefa 1:
Desenvolva um front-end que obtenha e apresente a previsão do tempo para todas as capitais da região
Sudeste.
## Dicas e Sugestões:
Desenvolva o código demonstrando seus conhecimentos quanto a modularização, organização e
padrões de projeto.
De preferência, utilizar o framework Angular para o desenvolvimento da solução.

#Entrega 
# i)  Entregue o código produzido através de arquivo compactado.
# ii) Entregue uma imagem apresentando a exibição das previsões da região sudeste no front-end.


In [14]:
!pip install xmltodict --quiet
!pip install flask_sqlalchemy --quiet
from urllib.request import urlopen
from xml.etree.ElementTree import parse
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
import pandas as pd
from tqdm import tqdm
import xmltodict
import seaborn as sea
import matplotlib.pyplot as plt
import ipywidgets as widgets


def tell_me_about(s): return (type(s), s)


def recupera_condicoes_meteorologicas():
  file = urlopen('http://servicos.cptec.inpe.br/XML/capitais/condicoesAtuais.xml')
  data = file.read()
  file.close()
  data = xmltodict.parse(data)
  capitais_  = {'SBRJ':'Rio de Janeiro', 
              'SBSP':'São Paulo', 
              'SBVT': 'Vitória', 
              'SBCF':'Belo Horizonte' }
  valores = []
  for cap in data['capitais']['metar']:  
    if cap['codigo'] in capitais_.keys():
      codigo = cap['codigo']
      atualizacao  = cap['atualizacao']
      pressao =  cap['pressao']
      temperatura=  cap['temperatura']      
      tempo=  cap['tempo']
      tempo_desc=  cap['tempo_desc']
      #print(tell_me_about(tempo_desc.decode("iso-8859-1")))
      tempo_desc = bytes(tempo_desc,'iso-8859-1').decode('utf-8')
      umidade=  cap['umidade']
      vento_dir=  cap['vento_dir']
      vento_int=  cap['vento_int']
      intensidade=  cap['intensidade']
      
      valores.append([capitais_[codigo],
                      atualizacao,
                      pressao,
                      temperatura,
                      sig_dic[tempo],
                      tempo_desc,
                      umidade,
                      vento_dir,
                      vento_int,
                      intensidade])
  #[('nome', 'Vitória'), ('uf', 'ES'), ('id', '246')])]
  df = pd.DataFrame(valores, columns=['codigo', 
                                            'atualizacao',
                                              'pressao',
                                              'temperatura',
                                              'tempo',
                                              'tempo_desc',
                                              'umidade',
                                              'vento_dir',
                                              'vento_int',
                                              'intensidade'])
  df['atualizacao'] = pd.to_datetime(df['atualizacao'])
  return df


def atualiza_banco(dados):
  for v in dados.values:  
    registro = registro_meteorologico(capital  = v[0],
                                    atualizacao = v[1],
                                    pressao = v[2],
                                    temperatura = v[3],                                    
                                    tempo_desc = v[5],
                                    umidade = v[6],
                                    vento_dir = v[7],
                                    vento_int = v[8],
                                    intensidade  = v[9])
    db.session.add(registro)
  db.session.commit()



In [15]:
siglas = """ec	Encoberto com Chuvas Isoladas
ci	Chuvas Isoladas
c	Chuva
in	Instável
pp	Poss. de Pancadas de Chuva
cm	Chuva pela Manhã
cn	Chuva a Noite
pt	Pancadas de Chuva a Tarde
pm	Pancadas de Chuva pela Manhã
np	Nublado e Pancadas de Chuva
pc	Pancadas de Chuva
pn	Parcialmente Nublado
cv	Chuvisco
ch	Chuvoso
t	Tempestade
ps	Predomínio de Sol
e	Encoberto
n	Nublado
cl	Céu Claro
nv	Nevoeiro
g	Geada
ne	Neve
nd	Não Definido
pnt	Pancadas de Chuva a Noite
psc	Possibilidade de Chuva
pcm	Possibilidade de Chuva pela Manhã
pct	Possibilidade de Chuva a Tarde
pcn	Possibilidade de Chuva a Noite
npt	Nublado com Pancadas a Tarde
npn	Nublado com Pancadas a Noite
ncn	Nublado com Poss. de Chuva a Noite
nct	Nublado com Poss. de Chuva a Tarde
ncm	Nubl. c/ Poss. de Chuva pela Manhã
npm	Nublado com Pancadas pela Manhã
npp	Nublado com Possibilidade de Chuva
vn	Variação de Nebulosidade
ct	Chuva a Tarde
ppn	Poss. de Panc. de Chuva a Noite
ppt	Poss. de Panc. de Chuva a Tarde
ppm	Poss. de Panc. de Chuva pela Manhã"""


sig_ = []
sig_dic = {}
tamanhos = []
for sig in siglas.split("\n"):  
  tup = sig.split("\t")
  sig_dic[tup[0]] = tup[1]
  tamanhos.append(len(tup[1]))
  sig_.append(tup)
sig_.sort()
df_sig  = pd.DataFrame(sig_, columns=['Sigla', 'Status'])
#df_sig.head(3)

In [16]:
recupera_condicoes_meteorologicas()

,codigo,atualizacao,pressao,temperatura,tempo,tempo_desc,umidade,vento_dir,vento_int,intensidade
0,Belo Horizonte,2021-09-23 10:00:00,1022,21,Predomínio de Sol,Predomínio de Sol,52,100,36,>10000
1,Rio de Janeiro,2021-09-23 10:00:00,1022,22,Predomínio de Sol,Predomínio de Sol,41,170,22,>10000
2,São Paulo,2021-09-23 10:00:00,1024,15,Predomínio de Sol,Predomínio de Sol,64,120,40,>10000
3,Vitória,2021-09-23 10:00:00,1023,22,Predomínio de Sol,Predomínio de Sol,52,200,54,>10000


In [17]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///meteorologico.db'
db = SQLAlchemy(app)

class registro_meteorologico(db.Model):
  __tablename__ = "registro_meteorologico"
  id = db.Column(db.Integer, primary_key=True)  
  capital = db.Column(db.String(30))
  atualizacao = db.Column(db.DateTime)
  pressao = db.Column(db.Integer)
  temperatura =   db.Column(db.Integer)
  tempo_desc = db.Column(db.String(30))
  umidade = db.Column(db.Integer)
  vento_dir  = db.Column(db.Integer)
  vento_int = db.Column(db.Integer)
  intensidade = db.Column(db.String(10))

  def __repr__(self):
    return str({'id': self.id, 
                'capital' : self.capital,
                'atualizacao': self.atualizacao.strftime("%Y/%m/%d, %H:%M:%S")})

import os.path

if os.path.isfile('meteorologico.db'):
  print ("File exist")
else:  
    db.create_all()
    print ("File created")

"""
@app.route('/')
def home():
    return render_template("index.html")

@app.route('/visualize')
def visualize():
  fig,ax = plt.subplots(figsize=(16.18, 10))
  ax=sea.set_style(style='darkgrid')
  filtra_cidade
  canvas=FigureCanvas(fig)
  img=io.BytesIO()
  fig.savefig(img)
  img.seek(0)
  return send_file(img, mimetype='img/png')
"""

File exist


/media/wesleyz/HD1Tera/anaconda3/envs/jupyter_env/lib/python3.9/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


'\n@app.route(\'/\')\ndef home():\n    return render_template("index.html")\n\n@app.route(\'/visualize\')\ndef visualize():\n  fig,ax = plt.subplots(figsize=(16.18, 10))\n  ax=sea.set_style(style=\'darkgrid\')\n  filtra_cidade\n  canvas=FigureCanvas(fig)\n  img=io.BytesIO()\n  fig.savefig(img)\n  img.seek(0)\n  return send_file(img, mimetype=\'img/png\')\n'

In [18]:
atualiza_banco(recupera_condicoes_meteorologicas())

In [8]:
#load_data_from_database().hist()

In [9]:
#filtra_cidade()

In [19]:
def load_data_from_database():
  valores = []

  for reg in registro_meteorologico.query.all():
    valores.append({'id':reg.id,
                    'capital'  : reg.capital,
                    'atualizacao' : reg.atualizacao.strftime("%Y/%m/%d, %H:%M:%S"),
                    'pressao' : reg.pressao,
                    'temperatura' : reg.temperatura,
                    'tempo_desc' : reg.tempo_desc,
                    'umidade' : reg.umidade,
                    'vento_dir' : reg.vento_dir,
                    'vento_int' : reg.vento_int,
                    'intensidade'  : reg.intensidade})
  return pd.DataFrame(valores)

def filtra_cidade(cidade=""):
  aux = load_data_from_database().drop_duplicates(subset=['capital','atualizacao'])
  if len(cidade)==0:
    plota_dados(aux)
    return aux
  else:
    plota_dados(aux[aux['capital']==cidade].tail(4).copy())
    plot_dados_modelos(aux[aux['capital']==cidade], cidade)

    return aux[aux['capital']==cidade].copy()

def plota_dados(dados):
  
  plt.figure(figsize=(9.08,5))
  sea.lineplot(data=dados.tail(12), x='atualizacao', y='temperatura', hue='capital')
  plt.ylabel('°C')
  plt.xlabel('Data')
  plt.title('Observatório Climático')
  plt.show()


def plot_dados_modelos(dados, cidade):
  for variavel in ['umidade', 'pressao', 'vento_int']:    
    sea.lmplot(data=dados[dados['capital']==cidade], x=variavel, y='temperatura')
    plt.ylabel('°C')
    plt.xlabel(variavel)
    plt.title('Observatório Climático: '+cidade)
    plt.show()
    
    sea.jointplot(data=dados[dados['capital']==cidade], x=variavel, y='temperatura', kind='kde')
    plt.ylabel('°C')
    plt.xlabel(variavel)
    plt.title('Observatório Climático: '+cidade)
    plt.show()
    print("-------------------------------------------------------------------")
  
   


wid_capital = widgets.Dropdown(name='capital')
wid_capital.options =[''] + list(load_data_from_database()['capital'].unique())
interactive_plot = widgets.interactive(filtra_cidade,cidade=wid_capital)

output = interactive_plot.children[-1]
output.layout.height = '350'

interactive_plot


interactive(children=(Dropdown(description='cidade', options=('', 'Belo Horizonte', 'Rio de Janeiro', 'São Pau…

In [13]:
db.metadata.tables['registro_meteorologico']


Table('registro_meteorologico', MetaData(), Column('id', Integer(), table=<registro_meteorologico>, primary_key=True, nullable=False), Column('capital', String(length=30), table=<registro_meteorologico>), Column('atualizacao', DateTime(), table=<registro_meteorologico>), Column('pressao', Integer(), table=<registro_meteorologico>), Column('temperatura', Integer(), table=<registro_meteorologico>), Column('tempo_desc', String(length=30), table=<registro_meteorologico>), Column('umidade', Integer(), table=<registro_meteorologico>), Column('vento_dir', Integer(), table=<registro_meteorologico>), Column('vento_int', Integer(), table=<registro_meteorologico>), Column('intensidade', String(length=10), table=<registro_meteorologico>), schema=None)